<a href="https://colab.research.google.com/github/stox74/hoyoung_park/blob/master/Step_2_%EC%B0%B8%EA%B3%A0%EC%82%AC%EC%A0%84%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%98%EC%97%AC_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D%EC%9D%84_%EC%8B%A4%EC%8B%9C%ED%95%9C%EB%8B%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive')

!pip install konlpy

import pandas as pd
import numpy as np
import re

from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from collections import Counter

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.feature_extraction.text import TfidfVectorizer

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
     |████████████████████████████████| 19.4MB 231kB/s 
     |████████████████████████████████| 2.3MB 46.8MB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [0]:
def remove_words(new_text):
    stop_words = ['스탁론', '탁론']
    for stopword in stop_words:
        if stopword in new_text:
            return False 
    return True
    
def check_len(str_1, len_num):
    if len(str_1) > len_num:
        return True 
    return False
  

def make_noun_doc(text):
    twitter = Okt()
    text_nouns = twitter.nouns(text)
    noun_count = Counter(text_nouns)
    str_result = ' '.join(text_nouns)
  
    return text_nouns, str_result, noun_count

def sentiment_analyst(df, len_num):
  
    df[0] = df['text']
    df[0] = df[0].drop_duplicates()
    df = df.dropna()
    df[0] = df[0].apply(lambda x: re.sub('[^가-힣0-9a-zA-Z\\s]', '', x))
    df[0] = df[0].apply(lambda x: ' '.join(x.split()))
  
    df['remove_word'] = df[0].apply(lambda x: remove_words(x))
    df = df[df['remove_word'] == True]
  
    #print(df)
    df['len_check'] = df[0].apply(lambda x: check_len(x, len_num))
    df = df[df['len_check'] == True]
    txt_list = df[0].values.tolist()
  
   # str, int , float, list, dict
    noun_list = []
    sentence = []
    noun_cnt = []
  
  #print(df)
  
    for i in range(len(txt_list)):
         
        text = txt_list[i]
        nouns, str_, noun_count = make_noun_doc(text)
    
        noun_list.append(nouns)
        sentence.append(str_)
        noun_cnt.append(noun_count)
    noun_count_df = (pd.DataFrame(noun_cnt)).T.fillna(0)
    noun_count_df['freq'] = noun_count_df.sum(axis=1)
    return sentence


def get_senti_score(text, dictionary):
    text_nouns, str_result, noun_count = make_noun_doc(text)
    print(text_nouns)
    sent_df = dictionary
    try:
        tmp_df = sent_df.loc[text_nouns] # text_nouns는 index리스트!
        tmp_df= tmp_df.dropna()
        print(tmp_df)
        result =  tmp_df[['pos', 'neg','neu_score','pos_score','neg_score']].sum().values.tolist()
        print(result)
    except:
        result = [0.0,0.0,0.0]
  
    return result

In [11]:
sent_df=pd.read_excel('gdrive/My Drive/채권_감성분석/01_Data/paper_1_bond_dicionary_ver_4.0.xlsx')
test_df = pd.read_excel('gdrive/My Drive/Data_House/news_data/sample_data.xlsx')

sent_df = sent_df.rename(columns={'Unnamed: 0':'Words'})
sent_df = sent_df.set_index('Words')
sent_df

,pos,neg,neu_score,pos_score,neg_score,sent_ratio
Words,,,,,,
가격,0.021719,0.045836,0,0,1,0.321503
가계,0.031967,0.035079,0,0,1,0.476796
가계부채,0.018008,0.019838,0,0,1,0.475826
가능성,0.072495,0.072945,0,0,1,0.498453
가운데,0.021340,0.024887,0,0,1,0.461634
...,...,...,...,...,...,...
매파,0.031841,0.397186,0,0,1,0.074216
호키쉬,0.031841,0.397186,0,0,1,0.074216
비둘기,0.274589,0.014619,0,1,0,0.949451


In [12]:
test_df['Text'] = test_df['text'].apply(lambda x: re.sub('[^가-힣0-9a-zA-Z\\s]', '', str(x)))
test_df['Text'] = test_df['Text'].apply(lambda x: ' '.join(x.split()))
result = test_df['Text'].apply(lambda x : get_senti_score(x,sent_df))
print(test_df['Date'])

score_df = pd.DataFrame(result.values.tolist(), columns = ['pos', 'neg','neu_score','pos_score','neg_score'])

test_df = test_df.reset_index()
result_df = pd.concat([test_df, score_df], axis=1)

result_df['Date'] = pd.to_datetime(result_df['Date'], format='%Y%m%d') 
result_df = result_df.set_index('Date')

result_df = result_df.groupby(['Date']).sum()
result_df['pos_neg_ratio'] = result_df['pos']/(result_df['pos'] + result_df['neg'])
#result_df['ratio_MA'] = result_df['pos_neg_ratio'].rolling(window=60).mean()
#result_df['ratio_MA'].plot()

['개인', '거래', '뜻', '시장', '최근', '투자가', '재테크', '수단', '각광', '초', '금리', '시대', '데', '스마트폰', '사용', '보편화', '상대', '수익률', '금리', '비', '대면', '투자', '수', '특성', '주목', '한국', '지난달', '세계', '최초', '금융', '법', '제정']
            pos       neg  neu_score  pos_score  neg_score  sent_ratio
Words                                                                 
거래     0.019104  0.014676        0.0        1.0        0.0    0.565551
시장     0.125380  0.106102        0.0        1.0        0.0    0.541641
최근     0.045041  0.047607        0.0        0.0        1.0    0.486153
금리     0.620281  0.590213        0.0        1.0        0.0    0.512420
금리     0.620281  0.590213        0.0        1.0        0.0    0.512420
투자     0.055711  0.043179        0.0        1.0        0.0    0.563359
한국     0.044282  0.037923        0.0        1.0        0.0    0.538674
지난달    0.040233  0.039656        0.0        1.0        0.0    0.503609
세계     0.025051  0.028127        0.0        0.0        1.0    0.471077
금융     0.065663  0.060078

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike


['은행', '평균', '대출', '금리', '역대', '최저', '치', '안심', '전환', '대출', '취급', '영향', '주택', '담보', '대출', '평균', '금리', '가장', '연', '를', '기록', '가운데', '전체', '가계', '대출', '평균', '금리', '개월', '연속', '다시', '진입', '한국은행', '발표', '중', '금융기관', '가중', '평균', '금리', '예금', '은행', '평균', '대출', '금리', '신규', '취급', '액', '기준']
            pos       neg  neu_score  pos_score  neg_score  sent_ratio
Words                                                                 
은행     0.139972  0.086133        0.0        1.0        0.0    0.619059
평균     0.021635  0.011435        0.0        1.0        0.0    0.654208
대출     0.144653  0.118442        0.0        1.0        0.0    0.549814
금리     0.620281  0.590213        0.0        1.0        0.0    0.512420
최저     0.029184  0.012509        0.0        1.0        0.0    0.699968
대출     0.144653  0.118442        0.0        1.0        0.0    0.549814
취급     0.012778  0.000000        0.0        1.0        0.0    1.000000
영향     0.041793  0.043387        0.0        0.0        1.0    0.490648
주택     

In [13]:
result_df

,index,Unnamed: 0,pos,neg,neu_score,pos_score,neg_score,pos_neg_ratio
Date,,,,,,,,
2020-01-01,78,78,33.353002,29.392666,0.0,193.0,92.0,0.531559
2020-01-02,58,58,6.265056,5.247598,0.0,48.0,6.0,0.544189
2020-01-03,189,189,77.189924,67.298211,0.0,544.0,227.0,0.534230
2020-01-04,110,110,39.972860,36.164730,0.0,281.0,149.0,0.525008
2020-01-05,468,468,85.367442,79.160020,0.0,655.0,310.0,0.518864
2020-01-06,2257,2257,306.532072,269.065314,0.0,2021.0,806.0,0.532546
2020-01-07,1790,1790,161.568809,141.033580,0.0,1060.0,334.0,0.533931
2020-01-08,1045,1045,93.487395,86.434788,0.0,637.0,307.0,0.519599
2020-01-09,1145,1145,133.906951,120.570647,0.0,940.0,399.0,0.526203


In [0]:
## 금리상승 관련 텍스트 

test_txt = '''1월 금통위 기준금리 1.25%로 동결. 인하 소수의견 2명
1월 금통위에서 기준금리는 1.25%로 동결되었다. 조동철, 신인석 위원이 인하 소수의견을
나타냈다. 통화정책방향문의 주요 내용 및 변경된 점은 다음과 같다. 1) 세계경제는 교역 부진이
이어지면서 성장세 둔화 지속. 국제금융시장은 미중 무역협상 진전 등으로 대체로 안정된 모습을
나타낸 가운데 중동 지역의 긴장 고조로 변동성 일시 확대. 앞으로 보호무역주의 및 지정학적
리스크의 전개 상황 등에 영향 받을 전망. 2) 국내경제는 부진이 일부 완화되는 움직임.
건설투자와 수출이 감소를 지속하였으나 설비투자가 소폭 증가하고 소비 증가세도 확대. 고용은
일부 개선 지속. 성장률은 11월 전망경로와 부합한 2%대 초반 수준 예상. 건설투자 조정이
이어지겠지만 수출과 설비투자 부진이 점차 완화되고 소비 증가세는 완만하게 확대 전망.
소비자물가는 석유류 가격 상승 등으로 0% 후반으로 높아짐. 근원인플레는 0%대 중반,
기대인플레는 1% 후반 유지. 앞으로 소비자물가는 11월 전망경로와 부합해 1% 내외,
근원인플레는 0%대 후반 예상. 3) 금융시장에서는 반도체 경기 회복 기대 등으로 주가 상승.
주택가격은 수도권 중심으로 높은 오름세. 가계대출은 증가 규모 확대

경기 개선 기미 보이는 가운데 금융안정도 고려
한은 총재 기자회견의 주요 내용은 다음과 같다. 1) 주택가격은 다양한 요인으로 결정되지만
저금리가 상승 일부 영향 미침. 2) 반도체 금년 중반 회복 전망 유지. 3) 통화정책만으로 구조적
문제 해결 어려움. 4) 최근 경기 관련 긍정적 지표 늘어남. 5) 현재 통화정책 완화적. 정부 부동산
정책 관련 금융안정도 고려. 6) 기축통화국보다는 금리 높게 운용
점차 물가안정 (인하)에서 금융안정 (동결)으로 무게중심 이동 전망
KB증권은 1월 금통위 Preview에서의 ‘인하 소수의견은 1명 또는 조동철 위원이 가세해 2명이
될 수도 있을 것이나, 11월 금통위 의사록에서 2명에 대한 기대도 다소 선반영되었기 때문에
시장 영향력은 만장일치 우려보다는 적을 것’이라는 전망을 유지한다.
‘디플레이션으로 실질금리가 높아 인하 필요’라는 비둘기파들의 의견은 향후 소비자물가
상승률이 1% 내외로 상승하면 실질금리가 0%에 접근하며 설득력이 감소할 전망이다. 금통위의
경기에 대한 판단은 이번 금통위에서 확인되었듯이 개선되고 있다.
반면 가계부채 등 금융안정에 대한 경계는 높아질 전망이다. 주택담보대출 증가율이 상승 전환한
가운데, 12월 은행권 가계대출 증가 규모는 통계작성 이후 최고치를 기록했다. 글로벌 내에서도
GDP대비 90%대의 가계부채 증가율은 상위권이다.
KB증권은 연간전망에서의 ‘통화정책이 물가안정에서 금융안정으로 무게중심이 이동하며 동결
가능성이 확대될 것’이라는 전망을 유지한다. 금리 반락 시 추격 매수보다는 차익실현, 리스크
관리 권고도 유지한다'''

In [14]:
test_txt = test_txt.replace('[^가-힣0-9a-zA-Z\\s]', ' ')
test_txt = ' '.join(test_txt.split())
result = get_senti_score(test_txt,sent_df)
result

['금통위', '기준금리', '로', '동결', '인하', '소수', '의견', '명', '금통위', '기준금리', '로', '동결', '동철', '신인', '석', '위원', '인하', '의견', '통화정책', '방향', '문의', '주요', '내용', '및', '변경', '점', '다음', '세계', '경제', '교역', '부진', '성장', '세', '둔화', '지속', '국제', '금융시장', '중', '무역', '협상', '진전', '등', '대체로', '안정', '모습', '가운데', '중동', '지역', '긴장', '고조', '변동성', '일시', '확대', '앞', '보호무역', '주의', '및', '지정학적', '리스크', '전개', '상황', '등', '영향', '전망', '국내', '경제', '부진', '일부', '완화', '움직임', '건설', '투자', '수출', '감소', '지속', '설비', '투자가', '폭', '증가', '소비', '증가', '세도', '확대', '고용', '일부', '개선', '지속', '성장', '률', '전망', '경로', '부합', '초반', '수준', '예상', '건설', '투자', '조정', '수출', '설비', '투자', '부진', '점차', '완화', '소비', '증가', '완만', '확대', '전망', '소비자', '물가', '유류', '가격', '상승', '등', '후반', '인플레', '중반', '인플레', '후반', '유지', '앞', '소비자', '물가', '전망', '경로', '부합', '내외', '인플레', '후반', '예상', '금융시장', '반도체', '경기', '회복', '기대', '등', '주가', '상승', '주택', '가격', '수도권', '중심', '가계', '대출', '증가', '규모', '확대', '경기', '개선', '기미', '가운데', '금융', '안정', '고려', '은', '총재', '기자회견', '주요', '내용', '다음', '주택', '가격', '요인', '

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike


[13.45127642603794, 13.11331192960661, 0.0, 82.0, 118.0]

In [0]:
## 금리 하락 뉘앙스 보고서 

test_txt2 ='''1 월 17 일 한국은행 금융통화위원회는 기준금리를 1.25%로 동결해 시장 예상에 부합한 가운데,
조동철, 신인석 위원이 금리 인하 소수의견을 제시해 소수의견이 지난해 11 월 1 명에서 2 명으로
늘어났다.
한은은 국내경제 부진이 일부 완화되는 움직임을 보였다고 진단했다. 점차 수출이 개선되고, 반도체
업황 개선이 경기 회복 기대를 높일 요인이라고 전망했다. 경기 진단에 대해서는 다소 매파적인
내용이었지만, 여전히 성장률 전망치가 11 월 전망하는 수준에 부합하는 수준이라고 언급한 점을
감안하면, 이는 여전히 잠재성장률을 하회하는 수준이다. 따라서 연내 경기 부양을 위한 추가 인하
가능성은 여전히 열려있으며, 이주열 총재 또한 통화 완화정책을 유지하겠다는 발언으로 가능성을
열어뒀다고 본다.
이를 감안할 때 2020 년 2 분기 중 25bp 금리 인하 전망을 유지한다. 경기 개선 정도는 기저 영향이
크고, 여전히 미중 무역불확실성이 높아 실물 경기 회복의 가능성은 낮다고 판단하기 때문이다. 또한
저물가, 기대인플레 하락을 우려하는 위원들이 소수의견을 계속해서 제시하고 있음을 감안할 때 적어도
올해 한차례 추가 인하는 있을 것으로 보며, 2 분기 중 단행할 것으로 전망한다.
한편, 이번 기자 간담회에서는 정부의 부동산 정책과 이에 걸맞는 통화 정책이 있을 수 있는가가 주요
주제라고 말해도 과언이 아닐만큼 부동산 규제에 대한 질문이 많았다. 이에 대해 이주열 총재는 금융
안정도 고려하겠으나, 현재 정부의 부동산 정책이 현재 통화정책의 완화기조와 상충된다고 생각하지
않는다고 대답했고, 통화정책은 경기 대응 거시정책이라는 점을 감안할 때, 구조적 경제 문제는 미시적
접근이 적합하다고 주장했다. 이를 감안할 때도 금리 인하 가능성은 여전히 열려있다고 본다.
결국 채권시장은 점차적으로 2 분기 금리 인하 가능성을 반영해 나갈 것으로 예상하나, 미중 무역 1 차
합의 이후의 심리지표 개선 기대도 상존하고 있어 1 분기까지 3 년물 금리는 1.4% 내외에서 등락을
보일 것으로 예상한다. 단, 1 분기 말부터는 금리 인하 가능성을 반영하며 완만한 하락세를 전망한다.'''

In [15]:
test_txt2 = test_txt2.replace('[^가-힣0-9a-zA-Z\\s]', ' ')
test_txt2 = ' '.join(test_txt2.split())
result2 = get_senti_score(test_txt2,sent_df)

['월', '일', '한국은행', '금융통화위원회', '기준금리', '로', '동결', '시장', '예상', '부합', '가운데', '동철', '신인', '석', '위원', '금리', '인하', '의견', '제시', '의견', '지난해', '월', '명', '명', '은', '국내', '경제', '부진', '일부', '완화', '움직임', '진단', '점차', '수출', '개선', '반도체', '업황', '개선', '경기', '회복', '기대', '요인', '전망', '경기', '진단', '대해', '다소', '파적', '내용', '성장', '률', '전망', '치가', '월', '전망', '수준', '부합', '수준', '언급', '점', '감안', '잠재', '성장', '률', '회하', '수준', '따라서', '연내', '경기', '부양', '위', '추가', '인하', '가능성', '이주열', '총재', '또한', '통화', '완화', '정책', '유지', '발언', '가능성', '감안', '때', '년', '분기', '중', '금리', '인하', '전망', '유지', '경기', '개선', '정도', '기저', '영향', '중', '무역', '불확실', '실물', '경기', '회복', '가능성', '판단', '때문', '또한', '물가', '인플레', '하락', '우려', '위원', '의견', '계속', '제시', '음', '감안', '때', '올해', '차례', '추가', '것', '분기', '중', '단행', '것', '전망', '한편', '이번', '기자', '간담', '회', '정부', '부동산', '정책', '이', '걸', '통화', '정책', '수', '주요', '주제', '말', '과언', '부동산', '규제', '대한', '질문', '이', '대해', '이주열', '총재', '금융', '안정', '고려', '현재', '정부', '부동산', '정책', '현재', '통화정책', '완화', '기조', '상충', '생각', '대답', '통화정책

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
